In [5]:
import torch

# Training data for XOR plus bias
x = torch.tensor([[0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 1]], dtype=float)
y = torch.tensor([[0], [1], [1], [0]], dtype=float)
w1 = torch.randn(3, 2, dtype=float, requires_grad=True)
w2 = torch.randn(2, 1, dtype=float, requires_grad=True)
b2 = torch.randn(1, 1, dtype=float, requires_grad=True)

lr = .5
for t in range(1000):
    y_pred = x.mm(w1).sigmoid().mm(w2).add(b2).sigmoid()
    ll = y * y_pred + (1 - y) * (1 - y_pred)
    loss = -ll.log().sum()
    print(t, loss.item())
    loss.backward()
    
    with torch.no_grad():
        # Update weights using SGD.
        w1 -= lr * w1.grad
        w2 -= lr * w2.grad
        b2 -= lr * b2.grad
        
        # Clear the gradients for the next iteration.
        w1.grad.zero_()
        w2.grad.zero_()
        b2.grad.zero_()

39115016308
171 0.5827974864178802
172 0.5720421979864722
173 0.5615796312770157
174 0.5514014678744499
175 0.5414995045350182
176 0.5318656690937591
177 0.5224920338704576
178 0.5133708268091903
179 0.5044944405735987
180 0.49585543980636043
181 0.4874465667472654
182 0.4792607453902369
183 0.47129108434573974
184 0.4635308785614766
185 0.4559736100412598
186 0.44861294768951265
187 0.441442746397103
188 0.4344570454731677
189 0.4276500665172696
190 0.4210162108166382
191 0.4145500563444151
192 0.40824635442663526
193 0.40210002613824436
194 0.3961061584815896
195 0.39026000039463915
196 0.38455695863052664
197 0.37899259354493053
198 0.3735626148231815
199 0.36826287717486866
200 0.36308937601997526
201 0.35803824318726585
202 0.35310574264266625
203 0.3482882662627226
204 0.34358232966588664
205 0.3389845681122786
206 0.3344917324807234
207 0.3301006853302545
208 0.3258083970518161
209 0.32161194211465977
210 0.317508495410826
211 0.3134953287001361
212 0.3095698071573034
213 0.3057

In [6]:
x.mm(w1).sigmoid().mm(w2).add(b2).sigmoid()

tensor([[0.0049],
        [0.9947],
        [0.9947],
        [0.0080]], dtype=torch.float64, grad_fn=<SigmoidBackward>)